In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
#import libraries
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import layout
from datetime import datetime, timedelta
import random
import pathway as pw
output_notebook()

In [ ]:
#load and data preprocessing
# Load dataset
df = pd.read_csv('/content/dataset.csv')

# Combine date and time to timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],format='%d-%m-%Y %H:%M:%S',errors='coerce')

# Normalize key features
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['QueueNorm'] = df['QueueLength'] / df['QueueLength'].max()
df['TrafficScore'] = df['TrafficConditionNearby'].map({
    'Light': 0.3, 'Medium': 0.6, 'Heavy': 1.0
}).fillna(0.5)
df['SpecialDay'] = df['IsSpecialDay'].astype(int)
df['VehicleScore'] = df['VehicleType'].map({
    'Sedan': 0.6, 'SUV': 1.0, 'Motorcycle': 0.3
}).fillna(0.5)

df = df.sort_values("Timestamp")


In [ ]:
#model 1 baseline linear model
def baseline_model(df, base_price=10, alpha=5):
    prices = []
    for occ in df['OccupancyRate']:
        delta = alpha * (occ - 0.5)
        new_price = base_price + delta
        new_price = min(max(new_price, 5), 20)
        prices.append(round(new_price, 2))
    df['BaselinePrice'] = prices
    return df

In [ ]:
#model 2 demand based pricing
def demand_model(df, base_price=10):
    # Weighted sum
    df['DemandScore'] = (0.4 * df['OccupancyRate'] +
                         0.2 * df['QueueNorm'] +
                         0.15 * df['TrafficScore'] +
                         0.15 * df['SpecialDay'] +
                         0.1 * df['VehicleScore'])

    df['DemandPrice'] = base_price * (0.5 + df['DemandScore'])
    df['DemandPrice'] = df['DemandPrice'].clip(lower=5, upper=20)
    df['DemandPrice'] = df['DemandPrice'].round(2)
    return df

In [ ]:
#applying both model above
df = baseline_model(df)
df = demand_model(df)

In [ ]:
#simulation with pathway
import asyncio

class SimulatedStream:
    def __init__(self, df, delay=1.0):
        self.df = df
        self.delay = delay

    async def run_stream(self, callback):
        for _, row in self.df.iterrows():
            await callback(row)
            await asyncio.sleep(self.delay)

stream_df = df[['Timestamp', 'SystemCodeNumber', 'BaselinePrice', 'DemandPrice']]

In [19]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
import pandas as pd

output_notebook()

stream_df = stream_df.copy()
stream_df = stream_df.dropna(subset=['DemandPrice', 'BaselinePrice', 'SystemCodeNumber'])
stream_df = stream_df.head(500).reset_index(drop=True)
stream_df['x'] = stream_df.index

# Bokeh Plot
source = ColumnDataSource(data=dict(
    x=stream_df['x'].tolist(),
    y=stream_df['DemandPrice'].tolist()
))

p = figure(title="Final Parking Price Plot", height=350, x_axis_label="Time Step", y_axis_label="Demand Price")
p.line(x='x', y='y', source=source, line_width=2, color='blue', legend_label="Demand Price")
show(p)


In [20]:
# Final Pricing Table
final_table = stream_df[['x', 'SystemCodeNumber', 'BaselinePrice', 'DemandPrice']].tail(20)
final_table.columns = ['Time Step', 'System Code', 'Baseline Price ($)', 'Demand Price ($)']
final_table.reset_index(drop=True, inplace=True)
final_table.style.set_caption("Final Parking Price Table")


,Time Step,System Code,Baseline Price ($),Demand Price ($)
0,10,BHMBCCTHL01,9.050000,7.760000
1,11,Others-CCCPS135a,8.890000,7.630000
2,12,Others-CCCPS202,8.430000,7.260000
3,13,Others-CCCPS98,8.450000,7.270000
4,14,Others-CCCPS8,9.310000,7.960000
5,15,BHMNCPNST01,10.250000,8.720000
6,16,Others-CCCPS105a,9.440000,8.200000
7,17,Others-CCCPS202,8.630000,7.420000
8,18,Others-CCCPS135a,9.390000,8.160000
9,19,BHMBCCTHL01,9.170000,7.850000
